# Chi sono gli atleti che presentano elementi più difficili?

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import plotly.express as px

In [2]:
df_tot = pd.read_csv("final.csv")
# per rispondere alle mie domande mi basta avere gli elements, quindi elimino le rige relative ai components
print("initial df.shape:",df_tot.shape)
df = df_tot.drop(df_tot[df_tot.section == "components"].index)
print("post cut df.shape:",df.shape)

initial df.shape: (214491, 19)
post cut df.shape: (136861, 19)


In [3]:
df.head()

,competition,program,performance_id,athlete_name,athlete_nation,rank,total_segment_score,total_element_score,total_component_score,total_deductions,aspect_id,aspect_desc,element_order,element_base_value,section,aspect_final_score,judge_name,judge_nation,judge_score
2,ISU European Figure Skating Championships 2017,LADIES SHORT PROGRAM,b639d77459,Evgenia MEDVEDEVA,RUS,1,78.92,42.0,36.92,0.0,38bda018f5,3Lo,4.0,5.61,elements,7.21,Agita ABELE,LAT,3.0
3,ISU European Figure Skating Championships 2017,LADIES SHORT PROGRAM,b639d77459,Evgenia MEDVEDEVA,RUS,1,78.92,42.0,36.92,0.0,50366b5d60,3F+3T,3.0,10.56,elements,12.16,Agita ABELE,LAT,3.0
4,ISU European Figure Skating Championships 2017,LADIES SHORT PROGRAM,b639d77459,Evgenia MEDVEDEVA,RUS,1,78.92,42.0,36.92,0.0,6e36e62b05,CCoSp4,6.0,3.50,elements,4.50,Agita ABELE,LAT,2.0
5,ISU European Figure Skating Championships 2017,LADIES SHORT PROGRAM,b639d77459,Evgenia MEDVEDEVA,RUS,1,78.92,42.0,36.92,0.0,8f7180d180,2A,5.0,3.63,elements,4.63,Agita ABELE,LAT,2.0
9,ISU European Figure Skating Championships 2017,LADIES SHORT PROGRAM,b639d77459,Evgenia MEDVEDEVA,RUS,1,78.92,42.0,36.92,0.0,c07f5eb3da,FCSp4,1.0,3.20,elements,4.20,Agita ABELE,LAT,2.0


## 2.Bubble chart base value e atleti

* Non è detto che chi fa gli elementi più difficili sia anche il migliore e li faccia bene (vedi rank e total_deductions)
* Pick a reasonable number of base values to make the plot more readable (95 percentile ?)
* Rifare grafico Bubble con nuova colonna "percentage_of_base_value" per discrimnare meglio le size delle bolle -> non ha funzionato

- In media l'atleta x ha portato eleemnti con base value di y
- asse x atleti, size è la media del base_value, rank medio dell'ateleta (forse?)
- 90% percentile sulla media dei base_value

In [4]:
# Creo una tabella che associa a ciascun atleta la media degli element_base_value e il suo rank medio
# la tabella alla fine sarà così composta:
# - athlete_name
# - category
# - avg_rank
# - avg_elem_base_value

# Elimino le colonne che non servono:
athlete_base_value = df.drop(['judge_name',
                               'judge_nation',
                               'judge_score',
                               'section',
                               'total_deductions',
                               'element_order',
                               'aspect_final_score',
                               'competition',
                               'athlete_nation',
                               'total_segment_score',
                               'total_element_score',
                               'total_component_score',
                               'performance_id'], axis=1)
# Eimino le righe di doppioni di aspect_id con i diversi voti dei giudici:
athlete_base_value.drop_duplicates()
athlete_base_value.head()


,program,athlete_name,rank,aspect_id,aspect_desc,element_base_value
2,LADIES SHORT PROGRAM,Evgenia MEDVEDEVA,1,38bda018f5,3Lo,5.61
3,LADIES SHORT PROGRAM,Evgenia MEDVEDEVA,1,50366b5d60,3F+3T,10.56
4,LADIES SHORT PROGRAM,Evgenia MEDVEDEVA,1,6e36e62b05,CCoSp4,3.50
5,LADIES SHORT PROGRAM,Evgenia MEDVEDEVA,1,8f7180d180,2A,3.63
9,LADIES SHORT PROGRAM,Evgenia MEDVEDEVA,1,c07f5eb3da,FCSp4,3.20


In [5]:
# new categories
conditions = [
    (athlete_base_value['program'] == "LADIES SHORT PROGRAM") |
    (athlete_base_value['program'] == "JUNIOR LADIES SHORT PROGRAM") |
    (athlete_base_value['program'] == "JUNIOR LADIES FREE SKATING") |
    (athlete_base_value['program'] == "LADIES FREE SKATING"),

    (athlete_base_value['program'] == "MEN SHORT PROGRAM") |
    (athlete_base_value['program'] == "JUNIOR MEN SHORT PROGRAM") |
    (athlete_base_value['program'] == "MEN FREE SKATING") |
    (athlete_base_value['program'] == "JUNIOR MEN FREE SKATING"),

    (athlete_base_value['program'] == "PAIRS SHORT PROGRAM") |
    (athlete_base_value['program'] == "JUNIOR PAIRS SHORT PROGRAM") |
    (athlete_base_value['program'] == "PAIRS FREE SKATING") |
    (athlete_base_value['program'] == "JUNIOR PAIRS FREE SKATING"),

    (athlete_base_value['program'] == "ICE DANCE SHORT DANCE") |
    (athlete_base_value['program'] == "JUNIOR ICE DANCE SHORT DANCE") |
    (athlete_base_value['program'] == "ICE DANCE FREE DANCE") |
    (athlete_base_value['program'] == "JUNIOR ICE DANCE FREE DANCE"),

    ]

# create a list of the values we want to assign for each condition
values = ['Ladies','Men','Pairs','Ice dance']

# create a new column and use np.select to assign values to it using our lists as arguments
athlete_base_value['category'] = np.select(conditions, values)

# elimino le colonne che non mi servono più
athlete_base_value.drop(['aspect_id',
                          'aspect_desc',
                          'program'], axis=1)

# display updated DataFrame
athlete_base_value.head()

,program,athlete_name,rank,aspect_id,aspect_desc,element_base_value,category
2,LADIES SHORT PROGRAM,Evgenia MEDVEDEVA,1,38bda018f5,3Lo,5.61,Ladies
3,LADIES SHORT PROGRAM,Evgenia MEDVEDEVA,1,50366b5d60,3F+3T,10.56,Ladies
4,LADIES SHORT PROGRAM,Evgenia MEDVEDEVA,1,6e36e62b05,CCoSp4,3.50,Ladies
5,LADIES SHORT PROGRAM,Evgenia MEDVEDEVA,1,8f7180d180,2A,3.63,Ladies
9,LADIES SHORT PROGRAM,Evgenia MEDVEDEVA,1,c07f5eb3da,FCSp4,3.20,Ladies


In [6]:
# Creo una tabella che associa a ciascun atleta la media degli element_base_value e il suo rank medio
# degli elementi che ha presentato (avg_element_base_value)
athlete_base_value = athlete_base_value.groupby(['athlete_name','category']).mean().reset_index()
athlete_base_value.sort_values(by='element_base_value', ascending=False,inplace=True)
athlete_base_value.rename(columns={'element_base_value': 'avg_elem_base_value', 'rank': 'avg_rank'}, inplace=True)
athlete_base_value.head()

,athlete_name,category,avg_rank,avg_elem_base_value
231,Nathan CHEN,Men,2.375000,7.534813
59,Boyang JIN,Men,3.932773,7.391765
300,Vincent ZHOU,Men,6.075000,7.147000
263,Shoma UNO,Men,1.712500,7.052313
315,Yuzuru HANYU,Men,1.800000,6.806167


In [7]:
# Ordino data secondo il rank in modo crescente:
athlete_base_value = athlete_base_value.sort_values(by=['avg_rank'], ascending=True)
# Create a new column with the inverse of the rank:
athlete_base_value['avg_rank_inv'] = 1/athlete_base_value['avg_rank']

In [8]:
athlete_base_value.head(10)

,athlete_name,category,avg_rank,avg_elem_base_value,avg_rank_inv
103,Evgenia MEDVEDEVA,Ladies,1.000000,5.012632,1.000000
33,Anastasia MISHINA / Vladislav MIRZOEV,Pairs,1.000000,4.694444,1.000000
36,Anastasia SKOPTCOVA / Kirill ALESHIN,Ice dance,1.000000,4.875000,1.000000
303,Wenjing SUI / Cong HAN,Pairs,1.273684,4.952632,0.785124
285,Tessa VIRTUE / Scott MOIR,Ice dance,1.285714,5.276786,0.777778
25,Aljona SAVCHENKO / Bruno MASSOT,Pairs,1.368421,4.860526,0.730769
6,Alena KOSTORNAIA,Ladies,1.388889,5.287778,0.720000
105,Gabriella PAPADAKIS / Guillaume CIZERON,Ice dance,1.428571,5.148214,0.700000
16,Alexandra TRUSOVA,Ladies,1.611111,5.750000,0.620690
22,Aliona SAVCHENKO / Bruno MASSOT,Pairs,1.673684,4.877895,0.597484


In [9]:
# 10° percentile di tutti i dati
athlete_base_value.avg_elem_base_value.quantile(0.1)

3.8484285714285726

In [10]:
# 10° percentile per categoria
print('10° percentile Ladies: ',athlete_base_value[athlete_base_value['category'] == 'Ladies'].avg_elem_base_value.quantile(0.1))
print('10° percentile Men: ',athlete_base_value[athlete_base_value['category'] == 'Men'].avg_elem_base_value.quantile(0.1))
print('10° percentile Pairs: ',athlete_base_value[athlete_base_value['category'] == 'Pairs'].avg_elem_base_value.quantile(0.1))
print('10° percentile Ice dance: ',athlete_base_value[athlete_base_value['category'] == 'Ice dance'].avg_elem_base_value.quantile(0.1))

10° percentile Ladies:  3.4560526315789493
10° percentile Men:  4.0123999999999995
10° percentile Pairs:  4.084121212121213
10° percentile Ice dance:  4.327142857142862


In [11]:
# 90° percentile per categoria
print('90° percentile Ladies: ',athlete_base_value[athlete_base_value['category'] == 'Ladies'].avg_elem_base_value.quantile(0.9))
print('90° percentile Men: ',athlete_base_value[athlete_base_value['category'] == 'Men'].avg_elem_base_value.quantile(0.9))
print('90° percentile Pairs: ',athlete_base_value[athlete_base_value['category'] == 'Pairs'].avg_elem_base_value.quantile(0.9))
print('90° percentile Ice dance: ',athlete_base_value[athlete_base_value['category'] == 'Ice dance'].avg_elem_base_value.quantile(0.9))

90° percentile Ladies:  4.936388888888887
90° percentile Men:  6.18972603735196
90° percentile Pairs:  4.811045660508894
90° percentile Ice dance:  5.083278061224488


In [12]:
data_90th_perc_category = athlete_base_value[((athlete_base_value['category'] == 'Ladies') &
                                              (athlete_base_value['avg_elem_base_value'] > 4.94)) |
                                             ((athlete_base_value['category'] == 'Men') &
                                              (athlete_base_value['avg_elem_base_value'] > 6.19)) |
                                             ((athlete_base_value['category'] == 'Pairs') &
                                              (athlete_base_value['avg_elem_base_value'] > 4.81)) |
                                             ((athlete_base_value['category'] == 'Ice dance') &
                                              (athlete_base_value['avg_elem_base_value'] > 5.08))
                                            ]

In [13]:
data_90th_perc_category.head()

,athlete_name,category,avg_rank,avg_elem_base_value,avg_rank_inv
103,Evgenia MEDVEDEVA,Ladies,1.000000,5.012632,1.000000
303,Wenjing SUI / Cong HAN,Pairs,1.273684,4.952632,0.785124
285,Tessa VIRTUE / Scott MOIR,Ice dance,1.285714,5.276786,0.777778
25,Aljona SAVCHENKO / Bruno MASSOT,Pairs,1.368421,4.860526,0.730769
6,Alena KOSTORNAIA,Ladies,1.388889,5.287778,0.720000


In [14]:
print('shape tabella completa: ',athlete_base_value.shape)
print('shape 90° perc per categoria: ',data_90th_perc_category.shape)

shape tabella completa:  (319, 5)
shape 90° perc per categoria:  (33, 5)


In [15]:
# grafico con solo il 90° percentile per categoria
bubble = px.scatter(
    data_90th_perc_category, y="athlete_name", x="avg_elem_base_value",
    color="category", size="avg_rank_inv", 
    title='Base values of the elements for each aspect',
    log_x=False, size_max=60,
    labels={"category": "Category","avg_rank": "Average rank"},
    hover_data={"avg_rank_inv":False, "avg_rank": True},
    category_orders={"category": ['Ladies','Men','Pairs','Ice dance']},
    )
bubble.update_traces(
    marker=dict(symbol="star", line=dict(width=0.5, color='DarkSlateGrey')),
    selector=dict(mode="markers")
    )
bubble.update_layout(height=1000)
bubble.show()

In [16]:
# grafico con tutti i dati
bubble = px.scatter(
    athlete_base_value, y="athlete_name", x="avg_elem_base_value",
    color="category", size="avg_rank_inv", 
    title='Base values of the elements for each aspect',
    log_x=False, size_max=60,
    labels={"category": "Category","avg_rank": "Average rank"},
    hover_data={"avg_rank_inv":False, "avg_rank": True},
    category_orders={"category": ['Ladies','Men','Pairs','Ice dance']},
    )
bubble.update_traces(
    marker=dict(symbol="star", line=dict(width=0.5, color='DarkSlateGrey')),
    selector=dict(mode="markers")
    )
bubble.update_layout(height=1000)
bubble.show()